# **Install Requirments**

In [ ]:
!pip install --upgrade transformers
!pip install -q datasets
!pip install evaluate
!pip install rouge_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

# **Read Data**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load your CSV
df = pd.read_csv("/content/TROCR-FINAL-RESULT.csv")
df = df[["Extracted Text", "Prescription"]].dropna().reset_index(drop=True)
df.columns = ["input_text", "target_text"]

# Split into train and test (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


# **Model Training**

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set the language for Arabic-English
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "en_XX"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
max_len = 128

def preprocess(example):
    model_input = tokenizer(example["input_text"], max_length=max_len, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["target_text"], max_length=max_len, padding="max_length", truncation=True)
    model_input["labels"] = labels["input_ids"]
    return model_input

train_tokenized = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
test_tokenized = test_dataset.map(preprocess, remove_columns=test_dataset.column_names)


Map:   0%|          | 0/988 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/247 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mbart-correction",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: queenrody24 (queenrody24-instacart) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.088700,0.100662
2,0.071700,0.075761
3,0.055400,0.069447
4,0.024100,0.069527
5,0.015300,0.078306


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1235, training_loss=0.3550454170326231, metrics={'train_runtime': 1635.5163, 'train_samples_per_second': 3.02, 'train_steps_per_second': 0.755, 'total_flos': 1338202631700480.0, 'train_loss': 0.3550454170326231, 'epoch': 5.0})

# **Model Prediction and Evaluation**

In [ ]:
from transformers import Text2TextGenerationPipeline
from nltk.translate.bleu_score import sentence_bleu
import evaluate

# Create generation pipeline
pipe = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)  # or device=-1 for CPU

# Generate predictions
preds = []
for text in test_df["input_text"]:
    generated = pipe(text, max_length=128, clean_up_tokenization_spaces=True)[0]["generated_text"]
    preds.append(generated)

test_df["predicted_text"] = preds

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

predictions = test_df["predicted_text"].tolist()
references = test_df["target_text"].tolist()

rouge_scores = rouge.compute(predictions=predictions, references=references)
print(rouge_scores)


{'rouge1': np.float64(0.5786197228519316), 'rouge2': np.float64(0.4624859199412193), 'rougeL': np.float64(0.5733919312983105), 'rougeLsum': np.float64(0.5711057456318516)}


In [ ]:
#from datasets import load_metric  # if not using `evaluate`, use this fallback
from rouge_score import rouge_scorer
import pandas as pd

# Instantiate scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Compute ROUGE-L for each prediction
scores = []
for ref, pred in zip(test_df["target_text"], test_df["predicted_text"]):
    try:
        score = scorer.score(ref, pred)
        scores.append(score["rougeL"].fmeasure)
    except:
        scores.append(0.0)

# Add to DataFrame
test_df["rougeL_score"] = scores

# Sort by score
sorted_df = test_df.sort_values(by="rougeL_score", ascending=False)

# Top 5 best examples
print("\n🟢 Top 5 BEST predictions:")
for _, row in sorted_df.head(5).iterrows():
    print(f"\nInput: {row['input_text']}")
    print(f"Target: {row['target_text']}")
    print(f"Prediction: {row['predicted_text']}")
    print(f"ROUGE-L Score: {row['rougeL_score']:.3f}")

# Bottom 5 worst examples
print("\n🔴 Top 5 WORST predictions:")
for _, row in sorted_df.tail(5).iterrows():
    print(f"\nInput: {row['input_text']}")
    print(f"Target: {row['target_text']}")
    print(f"Prediction: {row['predicted_text']}")
    print(f"ROUGE-L Score: {row['rougeL_score']:.3f}")



🟢 Top 5 BEST predictions:

Input: Gabapentin الغداء بعد Valproate كل ساعات 6
Target: Gabapentin بعد الغداء , Valproate كل ٦ ساعات
Prediction: Gabapentin بعد الغداء, Valproate كل ٦ ساعات
ROUGE-L Score: 1.000

Input: Ratitidine Morning only
Target: Ranitidine  Morning only
Prediction: Ranitidine Morning only
ROUGE-L Score: 1.000

Input: كليباماسيت يومياً مرة Ibuprofen كل 8 ساعات signature
Target: كليندامايسين مرة يومياً , Ibuprofen كل ٨ ساعات
Prediction: كليباماسيت مرة يومياً, Ibuprofen كل ٨ ساعات
ROUGE-L Score: 1.000

Input: Carbocisteine daily Twice Omeprazole Every hours 6
Target: Carbocisteine  Twice daily , Omeprazole  Every 6 hours
Prediction: Carbocisteine Twice daily, Omeprazole Every 6 hours
ROUGE-L Score: 1.000

Input: أملاح الأمام الزوم As needed done Preddinis Before meals
Target: أملاح الإماهة الفموية  As needed , Prednisolone  Before meals
Prediction: أملاح الإماهة الفموية As needed, Prednisolone Before meals
ROUGE-L Score: 1.000

🔴 Top 5 WORST predictions:

Input: ديبروني

# **Save model and tokenizer**

In [ ]:
# Save model and tokenizer locally in Colab
model.save_pretrained("/content/mbart-ocr-correction")
tokenizer.save_pretrained("/content/mbart-ocr-correction")


('/content/mbart-ocr-correction/tokenizer_config.json',
 '/content/mbart-ocr-correction/special_tokens_map.json',
 '/content/mbart-ocr-correction/sentencepiece.bpe.model',
 '/content/mbart-ocr-correction/added_tokens.json',
 '/content/mbart-ocr-correction/tokenizer.json')

In [ ]:
import shutil
shutil.make_archive("/content/mbart-ocr-correction", 'zip', "/content/mbart-ocr-correction")


'/content/mbart-ocr-correction.zip'

In [ ]:
from google.colab import files
files.download("/content/mbart-ocr-correction.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>